In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [3]:
from toolbox import models
from django.forms.models import model_to_dict
import numpy as np
import json
from django.contrib.gis.geos import GEOSGeometry, Polygon, MultiPolygon
import pandas as pd

In [4]:
sinks = models.SiekerSink.objects.all()
sink_diffs = models.SinkDifference.objects.all()


In [5]:
for s in sinks:
    s.centroid = s.geom4326.centroid
    s.save()

In [ ]:
for sd in sink_diffs:
    if len(sd.geom25833) > 1:
        print(sd.id, len(sd.geom25833))

In [12]:
s = sinks.get(pk=4)

In [14]:
len(s.geom25833)

9

In [6]:
def merge_multipolygon_parts(id):
    sink = models.SiekerSink.objects.get(pk=id)
    diff = models.SinkDifference.objects.get(pk=id)
    sink.geom_remaining = diff.geom25833
    sink.save()

In [79]:
for sd in sink_diffs:
    merge_multipolygon_parts_from_remaining(sd.id)

5330 before remaining 1
1
4 before remaining 8
15
10154 before remaining 4
2
5515 before remaining 17
183
10653 before remaining 46
101
5135 before remaining 6
3
110 before remaining 35
101
12278 before remaining 5
31
10439 before remaining 1
1
5553 before remaining 1
1
5334 before remaining 1
1
11 before remaining 16
118
10156 before remaining 1
1
7 before remaining 14
34
9 before remaining 1
1
10164 before remaining 1
1
13 before remaining 1
1
21 before remaining 1
5
5146 before remaining 3
5
10171 before remaining 2
7
1346 before remaining 35
179
5341 before remaining 1
2
16 before remaining 1
2
18 before remaining 1
1
19 before remaining 1
1
2640 before remaining 1
1
24 before remaining 1
4
84 before remaining 10
6
11157 before remaining 1
1
60 before remaining 10
18
10173 before remaining 1
1
9862 before remaining 1
11
25 before remaining 95
336
26 before remaining 11
155
27 before remaining 1
1
29 before remaining 2
1
10175 before remaining 1
1
9867 before remaining 1
3
37 before

In [78]:
def merge_multipolygon_parts_from_remaining(id):
    sink = models.SiekerSink.objects.get(pk=id)
    print(sink.id, 'before remaining', len(sink.geom_remaining))
    parts = list(sink.geom_remaining)
    merged = sink.geom_single

    remaining = [g for g in parts if not g.equals(merged)]

    changed = True
    while changed:
        changed = False
        new_remaining = []
        for g in remaining:
            try:
                if merged.touches(g) or merged.intersects(g):
                    merged = merged.union(g.buffer(.01))
                    changed = True
                    if merged.geom_type == 'MultiPolygon':
                        # Force to a single polygon if possible
                        merged = merged.unary_union
                else:
                    new_remaining.append(g)
            except Exception as e:
                new_remaining.append(g)
        remaining = new_remaining

    # Return merged polygon, and remaining as MultiPolygon (or None)
    if remaining:
        remaining_multipolygon = MultiPolygon(remaining)
    else:
        remaining_multipolygon = None

    print(len(merged))
    sink.geom_single = merged
    sink.geom_remaining = remaining_multipolygon
    sink.save()
    
    

In [20]:
for sd in sink_diffs:
    merge_multipolygon_parts_from_remaining(sd.id)

5330 before remaining 1
2


TypeError: Cannot set SiekerSink SpatialProxy (POLYGON) with value of type: <class 'django.contrib.gis.geos.collections.MultiPolygon'>

In [70]:

sink = models.SiekerSink.objects.get(pk=5330)
print(sink.id, 'before remaining', len(sink.geom_remaining))
parts = list(sink.geom_remaining)
merged = sink.geom_single

remaining = [g for g in parts if not g.equals(merged)]

    

5330 before remaining 1


In [71]:
remaining

In [72]:

changed = True
while changed:
    changed = False
    new_remaining = []
    for g in remaining:
        try:
            if merged.touches(g) or merged.intersects(g):
                merged = merged.union(g.buffer(0.01))
                changed = True
                if merged.geom_type == 'MultiPolygon':
                    # Force to a single polygon if possible
                    print('still Multipol')
                    merged = merged.unary_union
            else:
                new_remaining.append(g)
        except Exception as e:
            new_remaining.append(g)
    remaining = new_remaining

In [73]:
merged

In [47]:
merged = merged.unary_union
merged[1].geojson

'{ "type": "Polygon", "coordinates": [ [ [ 444139.505271603062283, 5802348.895661107264459 ], [ 444139.505271603062283, 5802350.895659442991018 ], [ 444141.505274262977764, 5802350.895659442991018 ], [ 444141.505274262977764, 5802348.895661107264459 ], [ 444139.505271603062283, 5802348.895661107264459 ] ] ] }'

In [ ]:

changed = True
while changed:
    changed = False
    new_remaining = []
    for g in remaining:
        try:
            if merged.touches(g) or merged.intersects(g):
                merged = merged.union(g)
                changed = True
                if merged.geom_type == 'MultiPolygon':
                    # Force to a single polygon if possible
                    merged = merged.unary_union
            else:
                new_remaining.append(g)
        except Exception as e:
            new_remaining.append(g)
    remaining = new_remaining

# Return merged polygon, and remaining as MultiPolygon (or None)
if remaining:
    remaining_multipolygon = MultiPolygon(remaining)
else:
    remaining_multipolygon = None

print(len(merged))
sink.geom_single = merged
sink.geom_remaining = remaining_multipolygon
sink.save()


In [74]:
def merge_multipolygon_parts(multipolygon_geom):
    parts = list(multipolygon_geom)
    if len(parts) <= 1:
        return parts[0], None  # already a single polygon

    merged = max(parts, key=lambda g: g.area)
    print(type(merged))
    remaining = [g for g in parts if not g.equals(merged)]

    changed = True
    while changed:
        changed = False
        new_remaining = []
        for g in remaining:
            try:
                if merged.touches(g) or merged.intersects(g):
                    merged = merged.union(g.buffer(.01))
                    changed = True
                else:
                    new_remaining.append(g)
            except Exception as e:
                new_remaining.append(g)
        remaining = new_remaining

    # Return merged polygon, and remaining as MultiPolygon (or None)
    if remaining:
        remaining_multipolygon = MultiPolygon(remaining)
    else:
        remaining_multipolygon = None

    return merged, remaining_multipolygon


In [20]:
for s in sinks:
    # print('Processing ', s.id)
    #if s.geom_single is not None:
        
        #try:
    if (len(s.geom25833) > 1):
        print(s.id, 'len', len(s.geom25833)) 
        merged, leftover = merge_multipolygon_parts(s.geom25833)
        s.geom_single = merged[0]
        s.geom_remaining = leftover
        s.save()
        print('saved id: ', s.id)
    elif len(s.geom25833) == 1:
        s.geom_single = s.geom25833[0]
        s.geom_remaining = None
        s.save()
  #          else:
   #             s.geom_single = None
    #            s.geom_remaining = None
    
        
       # except Exception as e:
       #     print(f"Failed at {s.id}: {e}")


10153 len 4
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  10153
10154 len 5
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  10154
4 len 9
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  4
5 len 2
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  5
5515 len 18
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  5515
10653 len 47
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  10653
110 len 36
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  110
12278 len 6
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  12278
10155 len 2
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  10155
10156 len 2
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  10156
10157 len 2
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  10157
7 len 15
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  7
9 len 2
<class 'django.contrib.gis.geos.polygon.Polygon'>
saved id:  9
10 len 

In [76]:
multi = list(h.geom25833)

In [82]:
biggest = get_biggest_geom(multi)

In [83]:
biggest

In [81]:

s = sinks[2]
s.area


97812.049

In [86]:

s.geom_single.area


97816.37509589817

In [87]:
s.geom25833.area

97812.04867670067

In [73]:
biggest = h.geom25833[index_biggest]
remaining = []
i = 1
for i in range(0, len(h.geom25833)):
    if i != index_biggest:
        try:
            biggest = GEOSGeometry.union(h.geom25833[i], biggest)
            print('geom',i, geom, '\n')
        except:
            remaining.append(geom)


geom 0 SRID=25833;MULTIPOLYGON (((428225.4841069008 5756778.933588355, 428225.4841069008 5756776.933590019, 428223.4841042409 5756776.933590019, 428223.4841042409 5756778.933588355, 428225.4841069008 5756778.933588355)), ((428183.4840510432 5756754.93360833, 428181.4840483833 5756754.93360833, 428181.4840483833 5756756.933606665, 428179.48404572345 5756756.933606665, 428179.48404572345 5756758.933605, 428175.4840404037 5756758.933605, 428175.4840404037 5756756.933606665, 428173.4840377438 5756756.933606665, 428173.4840377438 5756758.933605, 428169.48403242405 5756758.933605, 428169.48403242405 5756756.933606665, 428167.48402976413 5756756.933606665, 428167.48402976413 5756754.93360833, 428161.4840217845 5756754.93360833, 428161.4840217845 5756760.933603336, 428167.48402976413 5756760.933603336, 428167.48402976413 5756762.933601671, 428173.4840377438 5756762.933601671, 428173.4840377438 5756764.933600007, 428171.4840350839 5756764.933600007, 428171.4840350839 5756766.933598342, 428169.4

In [75]:
len(biggest)

13

In [67]:
biggest.area

6424.003196937834

In [88]:
all_s = models.SiekerSink.objects.all()

In [89]:
for s in all_s:
    s.geom4326 = s.geom_single.transform(4326, clone=True)
    s.save()